<a href="https://colab.research.google.com/github/kaixih/JAX101/blob/master/pjit_sharding_positional_vs_named.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*PositionalSharding and NamedSharding are two ways to express sharding. This colab demonstrates how they are used for the data sharding.*

In [62]:
import os

import functools
from typing import Optional

import numpy as np

import jax
import jax.numpy as jnp

In [63]:
import os
os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=8'

import jax
from jax.experimental import mesh_utils
from jax.sharding import PositionalSharding
from jax.sharding import Mesh
from jax.sharding import PartitionSpec
from jax.sharding import NamedSharding

P = PartitionSpec

In [64]:
# Create an array of random values:
x = jax.random.normal(jax.random.PRNGKey(0), (8192, 8192))

In [65]:
# Create a Sharding object to distribute a value across devices:
devices = mesh_utils.create_device_mesh((4, 2))

Positional Sharding for 4x2

In [66]:
sharding = PositionalSharding(devices)
y = jax.device_put(x, sharding.reshape(4, 2))
jax.debug.visualize_array_sharding(y)

                        
   CPU 0       CPU 1    
                        
                        
   CPU 2       CPU 3    
                        
                        
   CPU 4       CPU 5    
                        
                        
   CPU 6       CPU 7    
                        

Named Sharding for 4x2

In [67]:
mesh = Mesh(devices, axis_names=('a', 'b'))
y = jax.device_put(x, NamedSharding(mesh, P('a', 'b')))
jax.debug.visualize_array_sharding(y)

                        
   CPU 0       CPU 1    
                        
                        
   CPU 2       CPU 3    
                        
                        
   CPU 4       CPU 5    
                        
                        
   CPU 6       CPU 7    
                        

Positional Sharding for 2x4 (Transpose)

In [68]:
y = jax.device_put(x, sharding.T)
jax.debug.visualize_array_sharding(y)

                                    
                                    
  CPU 0    CPU 2    CPU 4    CPU 6  
                                    
                                    
                                    
                                    
                                    
  CPU 1    CPU 3    CPU 5    CPU 7  
                                    
                                    
                                    

Named Sharding for 2x4 (Transpose)

In [69]:
y = jax.device_put(x, NamedSharding(mesh, P('b', 'a')))
jax.debug.visualize_array_sharding(y)

                                    
                                    
  CPU 0    CPU 2    CPU 4    CPU 6  
                                    
                                    
                                    
                                    
                                    
  CPU 1    CPU 3    CPU 5    CPU 7  
                                    
                                    
                                    

Positional Sharding for 4x1 (Replicate second axis)

In [70]:
y = jax.device_put(x, sharding.replicate(axis=1))
jax.debug.visualize_array_sharding(y)

                         
         CPU 0,1         
                         
                         
         CPU 2,3         
                         
                         
         CPU 4,5         
                         
                         
         CPU 6,7         
                         

Named Sharding for 4x1 (Replicate second axis)

In [71]:
y = jax.device_put(x, NamedSharding(mesh, P('a', None)))
jax.debug.visualize_array_sharding(y)

                         
         CPU 0,1         
                         
                         
         CPU 2,3         
                         
                         
         CPU 4,5         
                         
                         
         CPU 6,7         
                         

Positional Sharding for 1x2 (Replicate first axis)

In [72]:
y = jax.device_put(x, sharding.replicate(axis=0))
jax.debug.visualize_array_sharding(y)

                        
                        
                        
                        
                        
CPU 0,2,4,6 CPU 1,3,5,7 
                        
                        
                        
                        
                        

Named Sharding for 1x2 (Replicate first axis)

In [73]:
y = jax.device_put(x, NamedSharding(mesh, P(None, 'b')))
jax.debug.visualize_array_sharding(y)

                        
                        
                        
                        
                        
CPU 0,2,4,6 CPU 1,3,5,7 
                        
                        
                        
                        
                        

Positional Sharding for 8x1 (Flatten all devices)

In [74]:
y = jax.device_put(x, sharding.reshape(8, 1))
jax.debug.visualize_array_sharding(y)

          CPU 0          
                         
          CPU 1          
                         
          CPU 2          
                         
          CPU 3          
                         
          CPU 4          
                         
          CPU 5          
                         
          CPU 6          
                         
          CPU 7          
                         

Named Sharding for 8x1 (Flatten all devices)

In [75]:
y = jax.device_put(x, NamedSharding(mesh, P(('a', 'b'), None)))
jax.debug.visualize_array_sharding(y)

          CPU 0          
                         
          CPU 1          
                         
          CPU 2          
                         
          CPU 3          
                         
          CPU 4          
                         
          CPU 5          
                         
          CPU 6          
                         
          CPU 7          
                         

Positional Sharding for 1x4 (Transpose and replicate)

In [76]:
y = jax.device_put(x, sharding.T.replicate(0))
jax.debug.visualize_array_sharding(y)

                                    
                                    
                                    
                                    
                                    
 CPU 0,1  CPU 2,3  CPU 4,5  CPU 6,7 
                                    
                                    
                                    
                                    
                                    

Named Sharding for 1x4 (Transpose and replicate)

In [77]:
y = jax.device_put(x, NamedSharding(mesh, P(None, 'a')))
jax.debug.visualize_array_sharding(y)

                                    
                                    
                                    
                                    
                                    
 CPU 0,1  CPU 2,3  CPU 4,5  CPU 6,7 
                                    
                                    
                                    
                                    
                                    